In [1]:
!pip -q install -U \
  langgraph \
  langchain-core \
  langchain-community \
  rank-bm25 \
  pandas \
  pyarrow \
  tqdm \
  transformers \
  sentencepiece \
  torch \
  huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 112.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 135.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 130.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
!pip -q install -U langgraph langchain-core rank-bm25 pandas pyarrow tqdm

import json, pickle, re
from pathlib import Path
from typing import TypedDict, List, Dict, Any, Literal, Optional

import numpy as np
import pandas as pd
import torch
from rank_bm25 import BM25Okapi

from langgraph.graph import StateGraph, END

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
ART_DIR = Path("/content/drive/MyDrive/uci_rag_chunks")
EMB_DIR = Path("/content/drive/MyDrive/uci_rag_embeddings")
#load paths as required
# chunks
chunks_df = pd.read_parquet(ART_DIR / "chunks.parquet")

# bm25 tokens
with open(ART_DIR / "bm25_tokens.pkl", "rb") as f:
    corpus_tokens = pickle.load(f)

# embeddings
dense_matrix = torch.from_numpy(
    np.load(EMB_DIR / "dense_np.npy")
)

print("Chunks:", len(chunks_df))
print("Dense shape:", dense_matrix.shape)

Chunks: 213133
Dense shape: torch.Size([213133, 384])


In [8]:
bm25 = BM25Okapi(corpus_tokens)

Loading LLM - Qwen3B

In [9]:
!pip -q install transformers accelerate sentencepiece
!pip -q uninstall -y bitsandbytes # Uninstalling first to ensure a clean re-install
!pip install -U bitsandbytes>=0.46.1

In [10]:
!pip -q uninstall -y torchvision

In [ ]:
!python -c "import importlib.util; print(importlib.util.find_spec('torchvision'))" #check for torchvision incongruency

None


In [12]:
import os
# force-disable vision stack
os.environ["DISABLE_TRANSFORMERS_IMAGE_TRANSFORMS"] = "1"
os.environ["TRANSFORMERS_NO_TORCHVISION"] = "1"
os.environ["HF_HUB_DISABLE_TELEMETRY"] = "1"

In [13]:
import importlib.util
print("torchvision installed?", importlib.util.find_spec("torchvision") is not None)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import bitsandbytes as bnb
print("torch:", torch.__version__, "cuda:", torch.cuda.is_available())
print("bitsandbytes:", bnb.__version__)

torchvision installed? False
torch: 2.10.0+cu128 cuda: True
bitsandbytes: 0.49.1


In [14]:
LLM_ID = "microsoft/Phi-3.5-mini-instruct"
tok = AutoTokenizer.from_pretrained(LLM_ID, use_fast=True)
llm = AutoModelForCausalLM.from_pretrained(
    LLM_ID,
    device_map="auto",
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
)
llm.eval()
print("Loaded:", LLM_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

This model config has set a `rope_parameters['original_max_position_embeddings']` field, to be used together with `max_position_embeddings` to determine a scaling factor. Please set the `factor` field of `rope_parameters`with this ratio instead -- we recommend the use of this field over `original_max_position_embeddings`, as it is compatible with most model architectures.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Loaded: microsoft/Phi-3.5-mini-instruct


Testing LLM Chat Capabilities

In [ ]:
def llm_chat(user_text: str, system_text: str, max_new_tokens: int = 250) -> str:
    messages = [
        {"role": "system", "content": system_text},
        {"role": "user", "content": user_text},
    ]
    prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tok(prompt, return_tensors="pt").to(llm.device)

    with torch.no_grad():
        out = llm.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,              # IMPORTANT: deterministic helps JSON
            temperature=None,
            top_p=None,
            repetition_penalty=1.05,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.eos_token_id,
        )

    # decode only newly generated tokens
    gen_ids = out[0][inputs["input_ids"].shape[1]:]
    return tok.decode(gen_ids, skip_special_tokens=True).strip()

JSON Reprocessing

In [ ]:
def safe_json(text: str):
    text = text.strip()
    match = re.search(r"```(?:json)?\s*({.*}|\\[.*\\])\s*```?", text, re.DOTALL)
    if match:
        text = match.group(1).strip()
    else:
        # Fallback to finding first '{' or '[' and last '}' or ']' for standalone JSON
        start_candidates = [i for i in (text.find("["), text.find("{")) if i != -1]
        if start_candidates:
            start = min(start_candidates)
            end_bracket = text.rfind("]")
            end_brace = text.rfind("}")
            end = max(end_bracket, end_brace)
            if end > start:
                text = text[start:end+1]
            else:
                # If start token found but no valid end, it's not valid JSON structure
                raise ValueError("No valid JSON end token found.")
        else:
            # If no start token and no markdown block, then it's not JSON
            raise ValueError("No JSON start token or markdown block found.")

    try:
        return json.loads(text)
    except Exception as e:
        # Re-raise with a more informative message including the problematic text snippet
        raise ValueError(f"Failed to parse JSON: {e}\nAttempted to parse: {text[:1200]}") from e

In [17]:
def json_with_retry(prompt: str, system: str, max_new_tokens: int = 350, tries: int = 2):
    last_raw = None
    for t in range(tries):
        raw = llm_chat(prompt, system, max_new_tokens=max_new_tokens)
        last_raw = raw
        try:
            return safe_json(raw)
        except Exception:
            # Repair mode: only feed the bad output
            prompt = (
                "Convert the following text into STRICT valid JSON ONLY.\n"
                "Output JSON only, nothing else.\n\n"
                f"{raw}"
            )
            system = "You are a JSON formatter."
    raise ValueError(f"Could not parse JSON after retries. Last output:\n{last_raw[:1200]}")

Query Encoder

In [18]:
from transformers import AutoTokenizer, AutoModel

device = "cuda" if torch.cuda.is_available() else "cpu"
ENCODER_ID = "sentence-transformers/all-MiniLM-L6-v2"

enc_tok = AutoTokenizer.from_pretrained(ENCODER_ID, use_fast=True)
enc_model = AutoModel.from_pretrained(ENCODER_ID).to(device)
enc_model.eval()

@torch.no_grad()
def mean_pool(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1)
    summed = (last_hidden_state * mask).sum(dim=1)
    counts = mask.sum(dim=1).clamp(min=1e-9)
    return summed / counts

@torch.no_grad()
def embed_query(query: str, max_length: int = 256) -> torch.Tensor:
    inputs = enc_tok([query], padding=True, truncation=True, max_length=max_length, return_tensors="pt").to(device)
    out = enc_model(**inputs).last_hidden_state
    pooled = mean_pool(out, inputs["attention_mask"])
    pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
    return pooled[0].detach().cpu()  # [D]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [19]:
def tokenize(text: str):
    return re.findall(r"[A-Za-z0-9']+", text.lower())
def hybrid_search(query: str, k: int = 8, bm25_k: int = 80, alpha: float = 0.6):
    bm_scores = bm25.get_scores(tokenize(query))

    top = np.argsort(bm_scores)[::-1][:bm25_k].copy()   # FIX 1

    qv = embed_query(query)                              # [D]
    cand = dense_matrix[top].contiguous()                # FIX 2
    dense_scores = (cand @ qv).numpy()

    bm = bm_scores[top]
    bm = (bm - bm.min()) / (bm.max() - bm.min() + 1e-8)

    combo = alpha * dense_scores + (1 - alpha) * bm
    order = np.argsort(combo)[::-1][:k]

    hits = []
    for oi in order:
        idx = int(top[oi])
        row = chunks_df.iloc[idx]
        hits.append({
            "score": float(combo[oi]),
            "dense": float(dense_scores[oi]),
            "bm25": float(bm[oi]),
            "source_id": int(row["chunk_id"]),
            "drug": row.get("drug", None),
            "condition": row.get("condition", None),
            "rating": row.get("rating", None),
            "text": row["text"],
        })
    return hits

Retrieval Test

In [20]:
hits = hybrid_search("dizziness and nausea after starting medication", k=3)
for h in hits:
    print("\n---", round(h["score"],4), "id", h["source_id"])
    print(h["text"][:280])


--- 0.7768 id 12287
"Had bad stomach aches after taking the medication. Also dizziness and nausea. After a while the side effects lessened."

--- 0.7768 id 139261
"Had bad stomach aches after taking the medication. Also dizziness and nausea. After a while the side effects lessened."

--- 0.672 id 163341
"Soon after starting Micardis I suffered from dizziness, nausea, irritability and I had no energy. It continued, then I started shortness of breath. Was told by my doctor it was not the medication. I went to a cardiologist and she kept me on it and put me on Imdur. Then severe he


RAG System Development

In [21]:
from typing import TypedDict, List, Dict, Any, Literal, Optional
class RAGState(TypedDict, total=False):
    query: str
    intent: Literal["extract", "summarize", "qa"]
    hits: List[Dict[str, Any]]
    extractions: List[Dict[str, Any]]
    validated: List[Dict[str, Any]]
    final_answer: str

In [22]:
def node_route(state: RAGState) -> RAGState:
    q = state["query"]
    out = llm_chat(
        user_text=f"Return exactly one token: extract, summarize, or qa.\nQuery: {q}",
        system_text="You are an intent router. Output exactly one token.",
        max_new_tokens=5
    ).lower().strip()

    if "summar" in out:
        intent = "summarize"
    elif out.startswith("qa"):
        intent = "qa"
    else:
        intent = "extract"
    return {"intent": intent}

In [23]:
def node_retrieve(state: RAGState) -> RAGState:
    hits = hybrid_search(state["query"], k=8, bm25_k=80, alpha=0.6)
    return {"hits": hits}

Strict JSON Extraction

In [24]:
EXTRACTION_SCHEMA_EXAMPLE = """
[
  {
    "side_effect": "nausea",
    "severity": "mild",
    "onset": null,
    "duration": null,
    "negated": false,
    "evidence": "short quote from the chunk",
    "source_id": 123
  }
]
""".strip()

def node_extract(state: RAGState) -> RAGState:
    try:
      evidence = "\n".join([f"[{h['source_id']}] {h['text'][:700]}" for h in state["hits"]])
      prompt = (
        "Return ONLY a valid JSON array. No prose. No markdown.\n"
        "Each element must follow this schema example:\n"
        f"{EXTRACTION_SCHEMA_EXAMPLE}\n\n"
        "Rules:\n"
        "- side_effect must be a symptom/adverse effect explicitly mentioned.\n"
        "- severity must be one of: mild, moderate, severe, unknown.\n"
        "- onset/duration can be null if not stated.\n"
        "- negated=true if the evidence explicitly denies the symptom.\n"
        "- evidence must be a short direct snippet.\n"
        "- source_id must match the bracketed id.\n"
        "- If no side-effects are supported, return []\n\n"
        f"QUERY: {state['query']}\n\n"
        f"EVIDENCE:\n{evidence}\n\n"
        "JSON:"
    )
      extractions = json_with_retry(prompt, "You output JSON only.", tries=2)
      return {"extractions": extractions}
    except Exception as e:
      print(e)
      return {"extractions": []}

In [25]:
def node_validate(state: RAGState) -> RAGState:
    evidence = "\n".join([f"[{h['source_id']}] {h['text']}" for h in state["hits"]])

    prompt = (
        "Validate the extracted items against the evidence.\n"
        "Return STRICT JSON ONLY: {pass: boolean, corrected: [same schema as input]}.\n"
        "Remove any item not supported by the evidence. Fix negated if wrong.\n\n"
        f"Extractions:\n{json.dumps(state.get('extractions', []), ensure_ascii=False)}\n\n"
        f"Evidence:\n{evidence}\n\nJSON only."
    )

    obj = json_with_retry(prompt, "You are a strict validator. Output JSON only.", max_new_tokens=2000, tries=2)
    corrected = obj.get("corrected", state.get("extractions", []))
    return {"validated": corrected}

In [26]:
def node_answer(state: RAGState) -> RAGState:
    validated = state.get("validated", [])
    if not validated:
        return {
            "final_answer": "I couldn’t find clear, supported side-effect evidence in the retrieved reviews for this query."
        }

    prompt = (
        "Write a concise, clinically grounded response using ONLY the validated items.\n"
        "Add citations like [source_id] after each claim.\n"
        "Do not invent side-effects or frequencies.\n\n"
        f"User query: {state['query']}\n"
        f"Validated items:\n{json.dumps(validated, ensure_ascii=False)}\n\n"
        "Answer:"
    )

    final = llm_chat(prompt, "You are a careful medical assistant. No hallucinations.", max_new_tokens=220)
    return {"final_answer": final}

In [27]:
g = StateGraph(RAGState)

g.add_node("route", node_route)
g.add_node("retrieve", node_retrieve)
g.add_node("extract", node_extract)
g.add_node("validate", node_validate)
g.add_node("answer", node_answer)

g.set_entry_point("route")
g.add_edge("route", "retrieve")
g.add_edge("retrieve", "extract")
g.add_edge("extract", "validate")
g.add_edge("validate", "answer")
g.add_edge("answer", END)

app = g.compile()
print("Graph compiled.")

Graph compiled.


In [ ]:
DEFAULT_QUERY = "What side effects do people report for disprin and how severe are they?"

try:
    QUERY = EXTERNAL_QUERY
except NameError:
    QUERY = DEFAULT_QUERY

print("Running query:", QUERY)

In [ ]:
result = app.invoke({"query": QUERY})
print(result["final_answer"])

People report experiencing mild aching joints as a side effect of Disprin, which is less severe than initially experienced but still causes some level of discomfort (severity: mild). Additionally, there have been reports of severe flushing occurring within the first few days of use, although the exact severity and duration were not specified (severity: unknown) [36290, 199451]. A case of purple skin on cheeks was also reported, described as being extremely painful, similar to a severe sunburn, indicating a more severe reaction (severity: severe) [136335]. It should be noted that these instances vary among individuals, and further investigation into specific patient histories would provide a clearer understanding of the frequency and overall impact of these side effects.

References:
- Source ID 36290: Personal account detailing management strategies for mild side effects.
- Source ID 136335: Report of
